In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz

--2022-10-18 23:32:23--  https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274079476 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.0-bin-hadoop2.tgz.1’

spark-3.3.0-bin-had 100%[===================>] 261.38M   371MB/s    in 0.7s    

2022-10-18 23:32:23 (371 MB/s) - ‘spark-3.3.0-bin-hadoop2.tgz.1’ saved [274079476/274079476]



In [ ]:
!tar xf spark-3.3.0-bin-hadoop2.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop2"

In [ ]:
!pip install petastorm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import findspark
findspark.init()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .config("spark.csci316.pandas.SVM", "false") \
    .getOrCreate()
spark

In [ ]:
#Read the csv file
data = spark.read.csv("/content/drive/MyDrive/cleanWithHeader.csv", inferSchema=True, header=True)

In [ ]:
from petastorm.spark import SparkDatasetConverter, make_spark_converter
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import math

from IPython import embed
from matplotlib import colors
from matplotlib import pyplot as plt
from sklearn import datasets
import numpy as np
from scipy.special import logsumexp
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

# tensorflow hub
import tensorflow_hub as hub
# tensor flow module
import tensorflow as tf
import tensorflow_probability as tfp

# matplotlib
from matplotlib import colors
from matplotlib import pyplot as plt

# word vectorizor
# first converts the text into a matrix of word counts
# then transforms these counts by normalizing them based on the term frequency
from sklearn.feature_extraction.text import TfidfVectorizer

# used to create word encoders
from sklearn import preprocessing

/usr/local/lib/python3.7/dist-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem


In [ ]:
data.show(10)

+-------+---------+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-------------------------+---------------+----------+-------------------------+
|     id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|default_ind|revol_util_impute

In [ ]:
#Re-arrange the dataset
data_reordered = data.select("id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","int_rate","installment","annual_inc",
                             "dti","delinq_2yrs","inq_last_6mths","open_acc","pub_rec","revol_bal","total_acc","out_prncp","out_prncp_inv",
                             "total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries",
                             "collection_recovery_fee","revol_util_imputed","last_pymnt_amnt_imputed","collections_12_mths_ex_med_imputed",
                             "policy_code_imputed","tot_coll_amt_imputed","tot_cur_bal_imputed","total_rev_hi_lim_imputed","pymnt_plan_index",
                             "application_type_index","acc_now_delinq_index","grade_index","purpose_index","home_ownership_index",
                             "emp_length_index","verification_status_index","sub_grade_index","term_index","initial_list_status_index", "default_ind")

In [ ]:
df_train, df_val = data_reordered.randomSplit([0.7, 0.3], seed=135)

df_train = df_train.repartition(2)
df_val = df_val.repartition(2)

In [ ]:
df_train.show()

+--------+---------+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-------------------------+---------------+----------+-------------------------+-----------+
|      id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|revol_util_imputed|last_pym

In [ ]:
print(f"train: {df_train.count()}, val: {df_val.count()}")

train: 598745, val: 256998


In [ ]:
train_dataset = df_train.toPandas()

In [ ]:
test_dataset = df_val.toPandas()

In [ ]:
x_train = train_dataset.iloc[:,0:42]
y_train = train_dataset.iloc[:,-1]

In [ ]:
x_test = test_dataset.iloc[:,0:42]
y_test = test_dataset.iloc[:,-1]

In [ ]:
from IPython import embed
from matplotlib import colors
from matplotlib import pyplot as plt
from sklearn import datasets
import numpy as np
import tensorflow as tf
import numpy as np

In [ ]:
class TFNaiveBayesClassifier:
    dist = None

    def fit(self, X, y):
        # Separate training points by class (nb_classes * nb_samples * nb_features)
        unique_y = np.unique(y)
        points_by_class = np.array([
            [x for x, t in zip(X, y) if t == c]
            for c in unique_y])

        # Estimate mean and variance for each class / feature
        # shape: nb_classes * nb_features
        mean, var = tf.nn.moments(tf.constant(points_by_class), axes=[1])

        # Create a 3x2 univariate normal distribution with the 
        # known mean and variance
        self.dist = tf.distributions.Normal(loc=mean, scale=tf.sqrt(var))

    def predict(self, X):
        assert self.dist is not None
        nb_classes, nb_features = map(int, self.dist.scale.shape)

        # Conditional probabilities log P(x|c) with shape
        # (nb_samples, nb_classes)
        cond_probs = tf.reduce_sum(
            self.dist.log_prob(
                tf.reshape(
                    tf.tile(X, [1, nb_classes]), [-1, nb_classes, nb_features])),
            axis=2)

        # uniform priors
        priors = np.log(np.array([1. / nb_classes] * nb_classes))

        # posterior log probability, log P(c) + log P(x|c)
        joint_likelihood = tf.add(priors, cond_probs)

        # normalize to get (log)-probabilities
        norm_factor = tf.reduce_logsumexp(
            joint_likelihood, axis=1, keep_dims=True)
        log_prob = joint_likelihood - norm_factor
        # exp to get the actual probabilities
        return tf.exp(log_prob)

In [ ]:
x_train.values.astype(np.float32)

array([[3.6178144e+07, 3.8879660e+07, 2.9400000e+04, ..., 9.0000000e+00,
        1.0000000e+00, 1.0000000e+00],
       [7.3451380e+06, 9.0072710e+06, 1.6000000e+04, ..., 1.4000000e+01,
        0.0000000e+00, 0.0000000e+00],
       [2.3750490e+06, 2.8372320e+06, 3.5000000e+04, ..., 2.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [4.6417864e+07, 4.9555632e+07, 3.9000000e+03, ..., 7.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.4818632e+07, 5.8389360e+07, 2.8000000e+03, ..., 2.2000000e+01,
        0.0000000e+00, 0.0000000e+00],
       [4.9923412e+07, 5.3292136e+07, 6.0000000e+03, ..., 6.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [ ]:
y_train.values.astype(np.float32)

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [ ]:
x_train = tf.convert_to_tensor(x_train)
y_train = tf.convert_to_tensor(y_train)

In [ ]:
if __name__ == '__main__':

    tf_nb = TFNaiveBayesClassifier()
    tf_nb.fit(x_train, y_train)
    
    # Create a regular grid and classify each point
    x_min, x_max = x_train[:, 0].min() - .5, x_train[:, 0].max() + .5
    y_min, y_max = x_train[:, 1].min() - .5, x_train[:, 1].max() + .5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 30),
                         np.linspace(y_min, y_max, 30))
    s = tf.Session()
    Z = s.run(tf_nb.predict(np.c_[xx.ravel(), yy.ravel()]))
    # Extract probabilities of class 2 and 3
    Z1 = Z[:, 1].reshape(xx.shape)
    Z2 = Z[:, 2].reshape(xx.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':


ValueError: ignored

In [ ]:
# # Plot
#     fig = plt.figure(figsize=(5, 3.75))
#     ax = fig.add_subplot(111)

#     ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1,
#                 edgecolor='k')
#     # Swap signs to make the contour dashed (MPL default)
#     ax.contour(xx, yy, -Z1, [-0.5], colors='k')
#     ax.contour(xx, yy, -Z2, [-0.5], colors='k')

#     ax.set_xlabel('Sepal length')
#     ax.set_ylabel('Sepal width')
#     ax.set_title('TensorFlow decision boundary')
#     ax.set_xlim(x_min, x_max)
#     ax.set_ylim(y_min, y_max)
#     ax.set_xticks(())
#     ax.set_yticks(())

#     plt.tight_layout()
#     fig.savefig('tf_iris.png', bbox_inches='tight')

In [ ]:
# predProbGivenText_df = pd.DataFrame(y_pred.numpy())
# predProbGivenText_df.head()